In [1]:
import sqlite3
import pandas as pd
import re
import pickle
import datetime
import pymorphy2
from tensorflow.keras.preprocessing.text import \
    Tokenizer  # Методы для работы с текстами и преобразования их в последовательности
import numpy as np  # Для работы с данными
import matplotlib.pyplot as plt  # Для вывода графиков
import os  # Для работы с файлами
import keras
from tensorflow.keras import utils  # Для работы с категориальными данными

from tensorflow.keras.models import Sequential  # Полносвязная модель
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout  # Слои для сети

from tensorflow.keras.layers import SpatialDropout1D, BatchNormalization, Embedding, Flatten, \
    Activation  # Слои для сети
from tensorflow.keras.preprocessing.text import \
    Tokenizer  # Методы для работы с текстами и преобразования их в последовательности
from tensorflow.keras.preprocessing.sequence import pad_sequences  # Метод для работы с последовательностями

from sklearn.preprocessing import LabelEncoder  # Метод кодирования тестовых лейблов
from sklearn.model_selection import train_test_split  # Для разделения выборки на тестовую и обучающую
# from google.colab import drive # Для работы с Google Drive
from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM, Dense, Dropout, Input
import warnings
warnings.filterwarnings('ignore')

In [2]:
df= pd.read_excel('датасет.xlsx')
df['text'] = df['Суть обращения']
df['goal'] = df['Тематика письма'] 
df_text = df[['text', 'goal']]
df_text = df_text.dropna()
df_text = df_text.drop_duplicates()
df_text.reset_index(drop=True, inplace=True)

In [3]:

    # Приводм данные к сbbтрочному виду
df_text['text'] = [x.lower() for x in df_text['text']]

for i in range(df_text['text'].shape[0]):
        df_text['text'][i] = ' '.join(re.sub("(@[A-Za-z0-9]+)", " ", df_text['text'][i]).split())
        df_text['text'][i] = ' '.join(re.sub("\[[^()]*\]", " ", df_text['text'][i]).split())
        df_text['text'][i] = ' '.join(re.sub("[A-Za-z0-9]", " ", df_text['text'][i]).split())
df_text['text'] = [x.lower() for x in df_text['text']]
    # Удаляем рабочие символы и знаки припенания
df_text['text'] = [re.sub(r'\W+', ' ', x) for x in df_text['text']]
    # Посмотрим сколько различных слов в нашем массиве
results = set()
df_text['text'].str.lower().str.split().apply(results.update)
print('Количество слов в массиве с обращениями', len(results))

df_text.columns = ['text', 'category']

texts = df_text['text'].values #Извлекаем данные всех текстов из столбца text


classes = list(df_text['category'].values) #Извлекаем соответствующие им значения классов (лейблов) столбца text

maxWordsCount = 60000 #Зададим максимальное количество слов/индексов, учитываемое при обучении текстов

print(df_text['category'].unique()) #Выводим все уникальные значения классов

nClasses = df_text['category'].nunique()+1  #Задаём количество классов, обращаясь к столбцу category и оставляя 

tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                      lower=True, split=' ', oov_token='unknown', char_level=False)

tokenizer.fit_on_texts(texts) #"Скармливаем" наши тексты, т.е. даём в обработку методу, который соберет словарь частотности


#Преобразовываем категории в векторы
encoder = LabelEncoder() # Вызываем метод кодирования тестовых лейблов из библиотеки sklearn
encoder.fit(classes) # Подгружаем в него категории из нашей базы
classesEncoded = encoder.transform(classes) # Кодируем категории
print(encoder.classes_)
#Формируем матрицу индексов по принципу Bag of Words
xAll = tokenizer.texts_to_matrix(texts) #Каждое слово из текста нашло свой индекс в векторе длиной maxWordsCount

yAll = utils.to_categorical(classesEncoded, nClasses) # И выводим каждый лейбл в виде вектора длиной 22, 
    # Получим список слов, входящих в наш токенизатор (при наличии установленного maxWordsCount)
    # с 1кой в позиции соответствующего класса и нулями
    # разбиваем все данные на обучающую и тестовую выборки с помощью метода train_test_split из библиотеки sklearn
xTrain, xVal, yTrain, yVal = train_test_split(xAll, yAll, test_size=0.2, shuffle = True)

Количество слов в массиве с обращениями 34904
['Касательно простоя вагонов' 'Касательно изменения условий перевозки'
 'Касательно перевозки личных вещей' 'Прочее'
 'О ходе выплонения плана погрузки' 'согласование заявок ГУ'
 'Об обеспечении доступа к системе ЭТРАН.'
 'Об оформлении вагонов в АС ЭТРАН'
 'Касательно согласования плана на перевозку'
 'Касательно выдачи технических условий' 'Перевозка на особых условиях'
 'о применении Тарифного руководства' 'Касательно организации перевозки'
 'Об ограничениях отгрузки пиломатериалов'
 'Касательно выдачи исходных данных' 'О предоставлении лимитов погрузки'
 'О заключении договора' 'О предоставления скидки'
 'Касательно отцепки вагонов' 'Выполнение поручений правительства РФ'
 'О предоставлении информации об осуществлении грузоперевозок'
 'По вопросу согласования проекта Соглашения']
['Выполнение поручений правительства РФ'
 'Касательно выдачи исходных данных'
 'Касательно выдачи технических условий'
 'Касательно изменения условий перевозки

In [4]:

    # Создаём полносвязную сеть
model01 = Sequential()
    # Входной полносвязный слой
model01.add(Dense(100, input_dim=maxWordsCount,
                      activation="relu"))
    # Слой регуляризации Dropout
model01.add(Dropout(0.4))
    # Второй полносвязный слой
model01.add(Dense(100, activation='relu'))
    # Слой регуляризации Dropout
model01.add(Dropout(0.4))
    # Третий полносвязный слой
model01.add(Dense(100, activation='relu'))
    # Слой регуляризации Dropout
model01.add(Dropout(0.4))
    # Выходной полносвязный слой
model01.add(Dense(nClasses, activation='softmax'))

model01.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    # Обучаем сеть на выборке
history = model01.fit(xTrain,
                          yTrain,
                          epochs=20,
                          batch_size=128,
                          validation_data=(xVal, yVal))

Epoch 1/20


2023-06-23 20:59:01.821862: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


25/25 [==============================] - 2s 51ms/step - loss: 2.9402 - accuracy: 0.1228 - val_loss: 2.5379 - val_accuracy: 0.1988
Epoch 2/20
25/25 [==============================] - 1s 34ms/step - loss: 2.3760 - accuracy: 0.2606 - val_loss: 1.9843 - val_accuracy: 0.5450
Epoch 3/20
25/25 [==============================] - 1s 30ms/step - loss: 1.8667 - accuracy: 0.4281 - val_loss: 1.3856 - val_accuracy: 0.6750
Epoch 4/20
25/25 [==============================] - 1s 30ms/step - loss: 1.3544 - accuracy: 0.5797 - val_loss: 0.9930 - val_accuracy: 0.7688
Epoch 5/20
25/25 [==============================] - 1s 30ms/step - loss: 0.9406 - accuracy: 0.7091 - val_loss: 0.7745 - val_accuracy: 0.7763
Epoch 6/20
25/25 [==============================] - 1s 31ms/step - loss: 0.7182 - accuracy: 0.7713 - val_loss: 0.6605 - val_accuracy: 0.8025
Epoch 7/20
25/25 [==============================] - 1s 30ms/step - loss: 0.5705 - accuracy: 0.8213 - val_loss: 0.6365 - val_accuracy: 0.8000
Epoch 8/20
25/25 [======

In [5]:
def predict2(text,model,tokenizer,encoder):
    word = tokenizer.texts_to_matrix([text])
    currPred = model.predict(word)
    currOut = np.argmax(currPred, axis=1)
    return encoder.inverse_transform(currOut)
predict2("информация дополнительная касательно перевозки",model01,tokenizer,encoder)

1/1 [==============================] - 0s 50ms/step


array(['Касательно изменения условий перевозки'], dtype='<U59')

In [6]:
df_text

,text,category
0,дополнительная информация по обращению клиента...,Касательно простоя вагонов
1,дополнительная информация по обращению клиента...,Касательно простоя вагонов
2,дополнительная информация по обращению клиента...,Касательно простоя вагонов
3,дополнительная информация по обращению клиента...,Касательно простоя вагонов
4,доработка вопрос клиента по станции уфимка не ...,Касательно простоя вагонов
...,...,...
3995,между администрацией губернатора псковской обл...,По вопросу согласования проекта Соглашения
3996,о научно техническом сотрудничестве между фонд...,По вопросу согласования проекта Соглашения
3997,между правительством санкт петербурга и правит...,По вопросу согласования проекта Соглашения
3998,между правительством рф и правительством кнр п...,По вопросу согласования проекта Соглашения


In [7]:
import random
pd.set_option("max_colwidth", 800)
df_pred1 = pd.DataFrame(columns = ['text','true','predicted','istrue'])
df_pred = pd.DataFrame(columns = ['text','true','predicted','istrue'])
for i in range(30):
    a=random.randint(1,4000)
    df2=df_pred1
    text = df_text.iloc[[a]]['text'].values[0]
    tru_val = df_text.iloc[[a]]['category'].values[0]
    pred_val = predict2(text,model01,tokenizer,encoder)
    if pred_val == tru_val:
        b=1
    else:
        b=0
    df_pred = df_pred.append(pd.DataFrame({'text':text,'true':tru_val,'predicted':pred_val,'istrue':b},index=[i]))
    
df_pred

1/1 [==============================] - 0s 8ms/step


,text,true,predicted,istrue
0,тема обращения обращение ооо ска вагон отправителя получателя текст письма добрый день на станции вихоревка всб с простаивает в ожидании отправления на ст забайкальск экспорт заб груженый полувагон ооо ска накл принять меры к отправлению вагона по назначению с уважением максим куреев проект железнодорожные перевозки ооо ска тел доб моб,Касательно согласования плана на перевозку,Касательно согласования плана на перевозку,1
1,о целевом обучении в соответствии с федеральным законом от июля г фз о государственной гражданской службе российской федерации указом президента российской федерации от февраля г о конкурсе на замещение вакантной должности государственной гражданской службы российской федерации и на основании протокола заседания конкурсной комиссии по подведению итогов конкурса на замещение вакантных должностей государственной гражданской службы в департаменте по молодежной,О заключении договора,О заключении договора,1
2,по ст алтайская не ставят вовремя вагоны под выгрузку например вагон июня не оформлен дата в ведомости выставляется не по уведомлению о завершении грузооперации а когда вагон уже забрали клиент сообщает что данная ситуация происходит неоднократно по данной станции вагоны попали тоже под простой из за того что того не подавали потом долго не забирали потом собственник выставляет простой сам клиент сообщает что не является ни го ни гп он плательщик тарифа но его не интересуют сейчас операции по вагонам он жалуется на работу станции т к такое отношение к клиентам станция проявляет неоднократно,Прочее,Прочее,1
3,с завода все что касается качества и сроков поставки как правильно принять товар правила оформления документов для тех у кого есть вопросы мы создали эту тему чтобы помочь вам разобраться в тонкостях отгрузки с завода производителя об отгрузки пиломатериалов на заводе пиломатериал всегда отгружается в соответствии с договорными условиями,Об ограничениях отгрузки пиломатериалов,Об ограничениях отгрузки пиломатериалов,1
4,тема обращения задержка в пути следования всб ж д отправителя получателя текст письма добрый день направляю файл с данными по задержке вагонов в пути следования прошу выслать в приложенной таблице номер присвоенного обращения с указанием наименования дороги задержки с уважением руководитель департамента организации перевозок зао евросиб спб тс новиков в а тел доп моб круглосуточный диспетчерский центр уведомление о конфиденциальности содержание настоящего сообщения включая контактную информацию является собственностью группы компаний евросиб и предназначено исключительно для рабочей переписки вся электронная переписка с использованием адресов электронной почты группы компаний евросиб считается рабочей перепиской и подлежит контролю со стороны группы компаний евросиб в целях защиты конф...,Касательно простоя вагонов,Касательно простоя вагонов,1
5,доработка в обр_ вагоны не оформлены и не отправлены все еще стоят просьба уточнить сроки решения вопроса добрый день накладные эы и эы простаивают по ст онохой вост сиб ждтел предмет обращения жалоба фио горяева татьяна юл да организация ооо реил легеон окпо вопрос добрый день просим помочь в отправке порожних платформ собственности ооо реил легеон платформы простаивают с г они очень нужны под погрузку срываются сроки отправки у отправителей которые их ждут до станции дозвониться не можем потому что в справочной дают только жд телефоны на данный момент этот вид связи нам не доступен телефон дата поступления запроса тип запроса сообщение,согласование заявок ГУ,согласование заявок ГУ,1
6,перевозка на особых условиях отличается от перевозки в контейнере тем что в контейнерах перевозка осуществляется только в пределах одной страны а в вагонах разных стран транспортировка на особых условиях это перевозка грузов в вагоне или контейнере которая отличается от обычной перевозки тем что при перевозке в вагонах или контейнерах используются специальные приспособления и устройства которые не позволяют грузу смещаться при движени